
<br>
================================================<br>
Segmenting the picture of greek coins in regions<br>
================================================<br>
This example uses :ref:`spectral_clustering` on a graph created from<br>
voxel-to-voxel difference on an image to break this image into multiple<br>
partly-homogeneous regions.<br>
This procedure (spectral clustering on an image) is an efficient<br>
approximate solution for finding normalized graph cuts.<br>
There are two options to assign labels:<br>
* with 'kmeans' spectral clustering will cluster samples in the embedding space<br>
  using a kmeans algorithm<br>
* whereas 'discrete' will iteratively search for the closest partition<br>
  space to the embedding space.<br>


In [ ]:
print(__doc__)

Author: Gael Varoquaux <gael.varoquaux@normalesup.org>, Brian Cheung<br>
License: BSD 3 clause

In [ ]:
import time

In [ ]:
import numpy as np
from distutils.version import LooseVersion
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import skimage
from skimage.data import coins
from skimage.transform import rescale

In [ ]:
from sklearn.feature_extraction import image
from sklearn.cluster import spectral_clustering

these were introduced in skimage-0.14

In [ ]:
if LooseVersion(skimage.__version__) >= '0.14':
    rescale_params = {'anti_aliasing': False, 'multichannel': False}
else:
    rescale_params = {}

load the coins as a numpy array

In [ ]:
orig_coins = coins()

Resize it to 20% of the original size to speed up the processing<br>
Applying a Gaussian filter for smoothing prior to down-scaling<br>
reduces aliasing artifacts.

In [ ]:
smoothened_coins = gaussian_filter(orig_coins, sigma=2)
rescaled_coins = rescale(smoothened_coins, 0.2, mode="reflect",
                         **rescale_params)

Convert the image into a graph with the value of the gradient on the<br>
edges.

In [ ]:
graph = image.img_to_graph(rescaled_coins)

Take a decreasing function of the gradient: an exponential<br>
The smaller beta is, the more independent the segmentation is of the<br>
actual image. For beta=1, the segmentation is close to a voronoi

In [ ]:
beta = 10
eps = 1e-6
graph.data = np.exp(-beta * graph.data / graph.data.std()) + eps

Apply spectral clustering (this step goes much faster if you have pyamg<br>
installed)

In [ ]:
N_REGIONS = 25

###########################################################################<br>
Visualize the resulting regions

In [ ]:
for assign_labels in ('kmeans', 'discretize'):
    t0 = time.time()
    labels = spectral_clustering(graph, n_clusters=N_REGIONS,
                                 assign_labels=assign_labels, random_state=42)
    t1 = time.time()
    labels = labels.reshape(rescaled_coins.shape)
    plt.figure(figsize=(5, 5))
    plt.imshow(rescaled_coins, cmap=plt.cm.gray)
    for l in range(N_REGIONS):
        plt.contour(labels == l,
                    colors=[plt.cm.nipy_spectral(l / float(N_REGIONS))])
    plt.xticks(())
    plt.yticks(())
    title = 'Spectral clustering: %s, %.2fs' % (assign_labels, (t1 - t0))
    print(title)
    plt.title(title)
plt.show()